In [12]:
import guidance
import re

guidance.llm = guidance.llms.OpenAI("gpt-4")

def parse_best(prosandcons, options):
    print(prosandcons)
    print(options)
    best = int(re.findall(r'Best=(\d+)', prosandcons)[0])
    return options[best]

create_plan = guidance('''{{#system~}}
You are a helpful assistant.
{{~/system}}
{{#block hidden=True}}
{{#user~}}
I want to {{goal}}.
{{~! generate potential options ~}}
Can you please generate one option for how to accomplish this?
Please make the option very short, at most one line.
{{~/user}}
{{#assistant~}}
{{gen 'options' n=5 temperature=1.0 stop='<|im_end|>' max_tokens=500}}
{{~/assistant}}
{{/block}}
{{~! generate pros and cons and select the best option ~}}
{{#block hidden=True}}
{{#user~}}
I want to {{goal}}.

Can you please comment on the pros and cons of each of the following options, and then pick the best option?
---{{#each options}}
Option {{@index}}: {{this}}{{/each}}
---
Please discuss each option very briefly (one line for pros, one for cons), and end by saying Best=X, where X is the best option.
{{~/user}}
{{#assistant~}}
{{gen 'prosandcons' stop='<|im_end|>' temperature=0.0 max_tokens=500}}
{{~/assistant}}
{{/block}}
{{#user~}}
I want to {{goal}}.
{{~! Create a plan }}
Here is my plan:
{{parse_best prosandcons options}}
Please elaborate on this plan, and tell me how to best accomplish it.
{{~/user}}
{{#assistant~}}
{{gen 'plan' max_tokens=500 stop='<|im_end|>'}}
{{~/assistant}}''')

out = create_plan(goal='read more books', parse_best=parse_best)
out

Option 0: 
Pros: Establishes a consistent reading habit and is easy to track.
Cons: May feel repetitive and lacks social interaction.
---
Option 1: 
Pros: Provides motivation, accountability, and social interaction.
Cons: May have less control over book choices and reading pace.
---
Option 2: 
Pros: Encourages consistent reading and offers a support system.
Cons: Limited book choices and potential scheduling conflicts.
---
Option 3: 
Pros: Enhances reading experience through discussions and motivation.
Cons: May require more time commitment and less personal book choice.
---
Option 4: 
Pros: Promotes regular reading and offers a sense of accountability.
Cons: Less control over book selection and possible time constraints.
---
Best=1 (Option 1 provides a good balance of motivation, accountability, and social interaction while still allowing for some personal choice in reading.)
['Set a daily reading goal, such as one chapter or 30 minutes every day.', 'Join a book club to stay motivated

In [10]:
print('\n'.join(['Option %d: %s' % (i, x) for i, x in enumerate(out['options'])]))

Option 0: Join a book club to stay motivated with monthly selections and discussions.
Option 1: Join a book club to read and discuss new titles monthly.
Option 2: Join a book club to stay motivated and on track with regular readings.
Option 3: Join a book club to stay motivated and discuss your reads with others.
Option 4: Set a goal to read for 20 minutes every day before bedtime.


In [11]:
print(out['prosandcons'])

Option 0: 
Pros: Encourages motivation and engagement through discussions.
Cons: Limited to monthly selections, may not cater to personal interests.
---
Option 1: 
Pros: Exposure to new titles and genres.
Cons: May not provide enough motivation to stay on track.
---
Option 2: 
Pros: Provides structure and motivation through regular readings.
Cons: Limited interaction with others, may not cover diverse genres.
---
Option 3: 
Pros: Motivation through discussions and shared experiences.
Cons: May not have a consistent reading schedule.
---
Option 4: 
Pros: Develops a daily reading habit and routine.
Cons: Lacks social interaction and discussions.

Best=4
